The below cell is a basic implementation of LSTM which has 3 layers and is trained with a single patient sensor data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load data from CSV
def load_data(filename):
    df = pd.read_csv(filename)
    timestamps = df.iloc[:, 0].values
    X = df.iloc[:, 2:-1].values  # Features (sensor values)
    y = df.iloc[:, -1].values    # Target variable
    return timestamps, X, y

# Define the filename
filename = '/Users/jasper/data30.csv' 

# Load data from CSV
timestamps, X, y = load_data(filename)

# Check the shape of X to understand its dimensions
print("Shape of X before reshaping:", X.shape)

# Reshape X to include timesteps
n_timesteps = 1  
X = X.reshape(X.shape[0], n_timesteps, X.shape[1])

# Check the shape of X after reshaping
print("Shape of X after reshaping:", X.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM RNN model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),  # Adjust input shape here
    Dense(32, activation='relu'),
    Dense(len(np.unique(y))+1, activation='softmax')  # Number of unique classes in y
])

# Compile the model using TensorFlow
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using TensorFlow
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Shape of X before reshaping: (81577, 4)
Shape of X after reshaping: (81577, 1, 4)
Epoch 1/10


/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2040/2040 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7067 - loss: 0.8998 - val_accuracy: 0.7547 - val_loss: 0.6959
Epoch 2/10
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7687 - loss: 0.6571 - val_accuracy: 0.7742 - val_loss: 0.6439
Epoch 3/10
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7751 - loss: 0.6348 - val_accuracy: 0.7667 - val_loss: 0.6550
Epoch 4/10
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7800 - loss: 0.6303 - val_accuracy: 0.7826 - val_loss: 0.6092
Epoch 5/10
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7836 - loss: 0.6142 - val_accuracy: 0.7842 - val_loss: 0.6039
Epoch 6/10
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7854 - loss: 0.6090 - val_accuracy: 0.8005 - val_loss: 0.5945
Epoch 7/10
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7902 - loss: 0.6039 - val_accuracy: 0.7781 - val_loss: 0.6210
Epoch 8/10
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7898 - loss: 0.6043 - val_accurac

We are going to find the best set of hyper-parameters using gridCV method

In [2]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

# Function to create the Keras model
def create_model(lstm_units=64, dense_units=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential([
        LSTM(lstm_units, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
        Dropout(dropout_rate),
        LSTM(64),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(len(np.unique(y))+1, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create KerasClassifier wrapper with default values for dense_units, dropout_rate, learning_rate, and lstm_units
keras_model = KerasClassifier(build_fn=create_model, lstm_units=64, dense_units=32, dropout_rate=0.2, learning_rate=0.001, verbose=0)

# Define hyperparameters and their search space
param_grid = {
    'lstm_units': [64, 128],
    'dense_units': [32, 64],
    'learning_rate': [0.001, 0.01],
    'dropout_rate': [0.2, 0.3],
    'batch_size': [32, 64]
}

# Perform grid search
print("Starting grid search...")
grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Grid search completed.")

# Get the best hyperparameters
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the model with the best hyperparameters
print("Training the best model...")
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)
print("Model training completed.")

# Evaluate the best model on the test set
print("Evaluating the best model on the test set...")
test_score = best_model.score(X_test, y_test)
print("Test set evaluation completed.")


Starting grid search...


/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When u

Grid search completed.
Training the best model...


/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model training completed.
Evaluating the best model on the test set...
Test set evaluation completed.


In [3]:
for i, params in enumerate(grid_search.cv_results_['params']):
    print(f"Parameters: {params}, Accuracy: {grid_search.cv_results_['mean_test_score'][i]}")

Parameters: {'batch_size': 32, 'dense_units': 32, 'dropout_rate': 0.2, 'learning_rate': 0.001, 'lstm_units': 64}, Accuracy: 0.7501722216631386
Parameters: {'batch_size': 32, 'dense_units': 32, 'dropout_rate': 0.2, 'learning_rate': 0.001, 'lstm_units': 128}, Accuracy: 0.7677020481453196
Parameters: {'batch_size': 32, 'dense_units': 32, 'dropout_rate': 0.2, 'learning_rate': 0.01, 'lstm_units': 64}, Accuracy: 0.7080338954711927
Parameters: {'batch_size': 32, 'dense_units': 32, 'dropout_rate': 0.2, 'learning_rate': 0.01, 'lstm_units': 128}, Accuracy: 0.7080338954711927
Parameters: {'batch_size': 32, 'dense_units': 32, 'dropout_rate': 0.3, 'learning_rate': 0.001, 'lstm_units': 64}, Accuracy: 0.7449472585698134
Parameters: {'batch_size': 32, 'dense_units': 32, 'dropout_rate': 0.3, 'learning_rate': 0.001, 'lstm_units': 128}, Accuracy: 0.746402927444319
Parameters: {'batch_size': 32, 'dense_units': 32, 'dropout_rate': 0.3, 'learning_rate': 0.01, 'lstm_units': 64}, Accuracy: 0.7080338954711927


In the next part, we are improving the model by adding more layers and tuning the hyper-parameters

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load data from CSV
def load_data(filename):
    df = pd.read_csv(filename)
    timestamps = df.iloc[:, 0].values
    X = df.iloc[:, 2:-1].values  # Features (sensor values)
    y = df.iloc[:, -1].values    # Target variable
    return timestamps, X, y

# Define the filename
filename = '/Users/jasper/data30.csv'  

# Load data from CSV
timestamps, X, y = load_data(filename)

# Data Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape X to include timesteps
n_timesteps = 1  # You can adjust this according to your data
X = X.reshape(X.shape[0], n_timesteps, X.shape[1])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM RNN model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(64, activation='relu'),  # Updated dense_units to 32
    Dropout(0.3),
    Dense(len(np.unique(y))+1, activation='softmax')  # Number of unique classes in y
])

# Compile the model using TensorFlow with the specified learning rate
model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early Stopping callback to prevent overfitting
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model using TensorFlow with the specified batch size
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/50


/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1020/1020 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7688 - loss: 0.6941 - val_accuracy: 0.8259 - val_loss: 0.4954
Epoch 2/50
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8275 - loss: 0.5052 - val_accuracy: 0.8478 - val_loss: 0.4479
Epoch 3/50
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8438 - loss: 0.4635 - val_accuracy: 0.8584 - val_loss: 0.4296
Epoch 4/50
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8478 - loss: 0.4468 - val_accuracy: 0.8607 - val_loss: 0.4188
Epoch 5/50
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8540 - loss: 0.4293 - val_accuracy: 0.8621 - val_loss: 0.4009
Epoch 6/50
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8573 - loss: 0.4210 - val_accuracy: 0.8639 - val_loss: 0.3890
Epoch 7/50
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8562 - loss: 0.4172 - val_accuracy: 0.8658 - val_loss: 0.3935
Epoch 8/50
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8591 - loss: 0.4099 - val_accurac

The accuracy obtained is around 87% for a single patients sensor data. But when the model was tested by feeding other patient's data, it performed poorly. So, We thought of training the model with around 32 patient data combined together as a single dataset. We reserved another 10 patients data for testing purposes.

In [6]:
import os

# Function to load data from multiple CSV files
def load_data_from_multiple_files(directory):
    timestamps_list = []
    X_list = []
    y_list = []

    # Iterate over each CSV file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            timestamps, X, y = load_data(file_path)  # Assuming load_data function is defined
            timestamps_list.append(timestamps)
            X_list.append(X)
            y_list.append(y)

    # Concatenate the lists
    timestamps_concatenated = np.concatenate(timestamps_list)
    X_concatenated = np.concatenate(X_list)
    y_concatenated = np.concatenate(y_list)

    return timestamps_concatenated, X_concatenated, y_concatenated

# Define the directory containing CSV files
directory = '/Users/jasper/Desktop/SIT764/sensor_files'

# Load data from multiple CSV files
timestamps, X, y = load_data_from_multiple_files(directory)

# Data Preprocessing (Standardization)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape X to include timesteps
n_timesteps = 1  # You can adjust this according to your data
X_reshaped = X_scaled.reshape(X_scaled.shape[0], n_timesteps, X_scaled.shape[1])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Define the LSTM RNN model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(64, activation='relu'),  # Updated dense_units to 32
    Dropout(0.3),
    Dense(len(np.unique(y))+1, activation='softmax')  # Number of unique classes in y
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early Stopping callback to prevent overfitting
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 184s 5ms/step - accuracy: 0.5824 - loss: 1.1323 - val_accuracy: 0.6048 - val_loss: 1.0407
Epoch 2/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 176s 5ms/step - accuracy: 0.6031 - loss: 1.0517 - val_accuracy: 0.6210 - val_loss: 0.9999
Epoch 3/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 179s 5ms/step - accuracy: 0.6112 - loss: 1.0256 - val_accuracy: 0.6295 - val_loss: 0.9789
Epoch 4/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 178s 5ms/step - accuracy: 0.6174 - loss: 1.0097 - val_accuracy: 0.6370 - val_loss: 0.9654
Epoch 5/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 176s 5ms/step - accuracy: 0.6216 - loss: 1.0018 - val_accuracy: 0.6418 - val_loss: 0.9547
Epoch 6/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 181s 5ms/step - accuracy: 0.6248 - loss: 0.9938 - val_accuracy: 0.6436 - val_loss: 0.9471
Epoch 7/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 190s 6ms/step - accuracy: 0.6271 - loss: 0.9885 - val_accuracy: 0.6453 - val_loss: 0.9422
Epoch 8/50
33659/33659 ━━━━━━━━━━━━━━━━━━━━ 191s 6ms/step - ac

The accuracy of the model is low (67%). But this time when the model was tested using unseen data, the accuracy was around 50% which is even low.

We test the model against unseen data

In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, accuracy_score
model.save('trained_model.h5')
# Load the trained model
loaded_model = load_model('trained_model.h5')

# Function to load new data
def load_new_data(filename):
    df = pd.read_csv(filename)
    X_new = df.iloc[:, 2:-1].values  # Features (sensor values)
    y_new = df.iloc[:, -1].values    # Target variable
    timestamp = df.iloc[:, 0].values
    return X_new, y_new,timestamp

# Define the filename for the new CSV file
filename_new = '/Users/jasper/Desktop/SIT764/test_files/p037.csv'  

# Load data from the new CSV file
X_new, y_new,timestamp = load_new_data(filename_new)

# Data Preprocessing for the new dataset
# Perform the same preprocessing steps as before
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

# Reshape X_new to include timesteps
X_new = X_new.reshape(X_new.shape[0], n_timesteps, X_new.shape[1])

# Make predictions using the loaded model on the entire dataset
predictions = loaded_model.predict(X_new)

# Convert predictions from one-hot encoding to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Evaluate the performance of the model on the new dataset
# Calculate accuracy
accuracy = accuracy_score(y_new, predicted_labels)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(y_new, predicted_labels)
print("Classification Report:")
print(report)


1807/1807 ━━━━━━━━━━━━━━━━━━━━ 2s 947us/step
Accuracy: 0.5470415224913495
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.93      0.79     32032
           1       0.00      0.00      0.00       506
           2       0.30      0.01      0.01     12508
           3       0.11      0.00      0.00      2676
           5       0.00      0.00      0.00      1807
           6       0.13      0.23      0.17      7915
           7       0.00      0.00      0.00       356

    accuracy                           0.55     57800
   macro avg       0.18      0.17      0.14     57800
weighted avg       0.47      0.55      0.46     57800



In [49]:
df = pd.DataFrame(predicted_labels)
output_directory = '/Users/jasper/Desktop/SIT764/test_files'
    # Specify the output filename for the CSV file
output_filename = 'sample.csv'
    
    # Concatenate the output directory and filename
output_filepath = os.path.join(output_directory, output_filename)
    
    # Save the DataFrame to a CSV file
df.to_csv(output_filepath, index=False)
    
    # Print message indicating successful saving
print(f"CSV file saved to: {output_filepath}")

CSV file saved to: /Users/jasper/Desktop/SIT764/test_files/sample.csv


By analysing the output labels and f1 score of each label, we could find that the learning of model has been affected due to certain labels (0). Hence we are going to try out a few things in the upcoming week: Remove unwanted data from the combined dataset (Label 0 is not required and hence we are going to remove it), While training the data we are not going to shuffle the data since we feel that there might be temporal patterns in our dataset which needs to be captured for better prediction, Increase the number of batch size and epochs.

This time we are going to remove the rows that contains label 0

This time while training the model, we are going to make sure that the model knows the number of instances of a particular label so that it doesn't depend more on a single label

In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

# Function to load data
def load_data(filename):
    df = pd.read_csv(filename)
    X_new = df.iloc[:, :-1].values  # Features (sensor values)
    y_new = df.iloc[:, -1].values    # Target variable
    timestamp = df.iloc[:, 0].values
    # Filter out rows with target label 0
    non_zero_indices = np.where(y_new != 0)[0]
    timestamps_filtered = timestamp[non_zero_indices]
    X_filtered = X_new[non_zero_indices]
    y_filtered = y_new[non_zero_indices]
    return X_filtered, y_filtered, timestamps_filtered


# Load data from CSV file
filename = '/Users/jasper/Desktop/SIT764/test_files/p040.csv'
X, y, timestamps = load_data(filename)

# Data Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight('balanced', classes = np.unique(y_train),y = y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)
# Define the LSTM RNN model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_train)), activation='softmax')  # Update to match the number of unique labels (6)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early Stopping callback to prevent overfitting
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), 
                    callbacks=[early_stopping], class_weight=class_weights_dict)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/50


/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4526 - loss: 1.3833 - val_accuracy: 0.4950 - val_loss: 1.1393
Epoch 2/50
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5417 - loss: 1.0502 - val_accuracy: 0.6085 - val_loss: 1.0630
Epoch 3/50
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6388 - loss: 0.9024 - val_accuracy: 0.6743 - val_loss: 0.9156
Epoch 4/50
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6826 - loss: 0.7875 - val_accuracy: 0.7199 - val_loss: 0.7812
Epoch 5/50
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7259 - loss: 0.6599 - val_accuracy: 0.7537 - val_loss: 0.6637
Epoch 6/50
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7398 - loss: 0.5936 - val_accuracy: 0.7621 - val_loss: 0.6292
Epoch 7/50
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7667 - loss: 0.5427 - val_accuracy: 0.8052 - val_loss: 0.5489
Epoch 8/50
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7904 - loss: 0.4913 - val_accuracy: 0.8044 - val_

For one patient's data, the model's accuracy has increased from 87 to 90

In [2]:
import os
import pandas as pd
import numpy as np
def load_new_data(filename):
    df = pd.read_csv(filename)
    X_new = df.iloc[:, :-1].values  # Features (sensor values)
    y_new = df.iloc[:, -1].values    # Target variable
    timestamp = df.iloc[:, 0].values
    return X_new, y_new,timestamp
    
def load_data_from_multiple_files(directory):
    timestamps_list = []
    X_list = []
    y_list = []

    # Iterate over each CSV file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            X_new, y_new, timestamps = load_new_data(file_path)  # Load data from each CSV file

            # Filter out rows with target label 0
            non_zero_indices = np.where(y_new != 0)[0]
            timestamps_filtered = timestamps[non_zero_indices]
            X_filtered = X_new[non_zero_indices]
            y_filtered = y_new[non_zero_indices]

            timestamps_list.append(timestamps_filtered)
            X_list.append(X_filtered)
            y_list.append(y_filtered)

    # Concatenate the lists
    timestamps_concatenated = np.concatenate(timestamps_list)
    X_concatenated = np.concatenate(X_list)
    y_concatenated = np.concatenate(y_list)

    return timestamps_concatenated, X_concatenated, y_concatenated

# Define the directory containing CSV files
directory = '/Users/jasper/Desktop/SIT764/sensor_files'

# Load data from multiple CSV files
timestamps, X, y = load_data_from_multiple_files(directory)


In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape X to include timesteps
n_timesteps = 1  # You can adjust this according to your data
#X_reshaped = X_scaled.reshape(X_scaled.shape[0], n_timesteps, X_scaled.shape[1])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight('balanced', classes = np.unique(y_train),y = y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)
# Define the LSTM RNN model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_train)), activation='softmax')  # Update to match the number of unique labels (6)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early Stopping callback to prevent overfitting
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), 
                    callbacks=[early_stopping], class_weight=class_weights_dict)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

/Users/jasper/Desktop/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 151s 9ms/step - accuracy: 0.5068 - loss: 1.7331 - val_accuracy: 0.5920 - val_loss: 1.0300
Epoch 2/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 159s 10ms/step - accuracy: 0.5914 - loss: 1.3307 - val_accuracy: 0.6190 - val_loss: 0.9558
Epoch 3/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 151s 9ms/step - accuracy: 0.6245 - loss: 1.2093 - val_accuracy: 0.6555 - val_loss: 0.8809
Epoch 4/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 155s 9ms/step - accuracy: 0.6407 - loss: 1.1493 - val_accuracy: 0.6602 - val_loss: 0.8552
Epoch 5/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 144s 9ms/step - accuracy: 0.6494 - loss: 1.1103 - val_accuracy: 0.6698 - val_loss: 0.8332
Epoch 6/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 167s 10ms/step - accuracy: 0.6581 - loss: 1.0853 - val_accuracy: 0.6817 - val_loss: 0.8016
Epoch 7/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 141s 8ms/step - accuracy: 0.6643 - loss: 1.0602 - val_accuracy: 0.6791 - val_loss: 0.8066
Epoch 8/50
16767/16767 ━━━━━━━━━━━━━━━━━━━━ 151s 9ms/step - 

The accuracy has improved from 66 to 73.5%